In [ ]:
from astropy.io import ascii
import astropy.units as u
from astropy.constants import G
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.integrate import simps

import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic
import biff.scf as bscf

from barchaos.potential import Config
from barchaos.potential import get_bar_potential

In [ ]:
tbl = np.genfromtxt('../data/MW-Menc.txt', names=True, delimiter=',')
sofue = ascii.read('../data/Sofue.dat')

In [ ]:
c = Config()
c.Omega = 0.
c.bar_mass = 1.32E9

In [ ]:
pot_no_halo = gp.CCompositePotential()
pot_no_halo['disk'] = gp.MiyamotoNagaiPotential(m=6E10*u.Msun,
                                        a=3*u.kpc,
                                        b=280*u.pc,
                                        units=galactic)

pot_no_halo['disk2'] = gp.MiyamotoNagaiPotential(m=2E10*u.Msun,
                                         a=6.*u.kpc,
                                         b=130*u.pc,
                                         units=galactic)

pot_no_halo['antidisk'] = gp.MiyamotoNagaiPotential(m=-2E10, 
                                                    a=2., b=0.28, 
                                                    units=galactic)

pot_no_halo['SgrA*'] = gp.KeplerPotential(m=4E6*u.Msun,
                                          units=galactic)

pot_no_halo['nucl'] = gp.HernquistPotential(m=9E7*u.Msun,
                                            c=10*u.pc,
                                            units=galactic)

pot_no_halo['spher'] = gp.HernquistPotential(m=6E9*u.Msun,
                                        c=0.25*u.kpc,
                                        units=galactic)
pot_no_halo['bar'] = get_bar_potential()

pot = pot_no_halo.copy()

pot['halo'] = gp.NFWPotential(m=6.67E11, r_s=20*u.kpc,
                              units=galactic)

In [ ]:
(pot['bar'].mass_enclosed([10,0,0.]),
 pot.mass_enclosed([250.,0,0.]),
 pot.mass_enclosed([8.,0,0.]))

In [ ]:
np.sqrt(G * 9.01047988e+10*u.Msun / (8*u.kpc)).to(u.km/u.s)

In [ ]:
pot.circular_velocity([8.,0,0.])

In [ ]:
xyz = np.zeros((3, 256))
xyz[0] = np.logspace(-3, 2.5, xyz.shape[1])

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].loglog(xyz[0], pot.mass_enclosed(xyz).value, marker='', lw=2.)
axes[0].set_xlim(2E-3, 250.)
axes[0].tick_params(which='both', right=True, top=True)
axes[0].errorbar(tbl['r'], tbl['Menc'], yerr=(tbl['Menc_neg_err'], tbl['Menc_pos_err']),
                 marker='o', ls='none')

axes[1].plot(xyz[0], pot.circular_velocity(xyz).to(u.km/u.s), marker='', lw=2.)
axes[1].plot(sofue['Radius'], sofue['Velocity'])

axes[1].plot(xyz[0], pot['halo'].circular_velocity(xyz).to(u.km/u.s), 
             marker='', lw=1., alpha=0.5)

disk_vc = np.sqrt(G * (pot['disk'].mass_enclosed(xyz) + 
                       pot['disk2'].mass_enclosed(xyz) + 
                       -pot['antidisk'].mass_enclosed(xyz)) / (xyz[0]*u.kpc)).to(u.km/u.s)
axes[1].plot(xyz[0], disk_vc, marker='', lw=1., alpha=0.5)
axes[1].plot(xyz[0], np.sqrt(pot['bar'].circular_velocity(xyz)**2 + 
                             pot['spher'].circular_velocity(xyz)**2 + 
                             pot['nucl'].circular_velocity(xyz)**2).to(u.km/u.s), 
             marker='', lw=1., alpha=0.5)

axes[1].axhline(220.)
axes[1].axvline(8.)
axes[1].fill_between([0,20], 200., 240, alpha=0.4, color='#888888')
axes[1].set_xlim(0, 20)
axes[1].set_ylim(50, 280)
axes[1].set_xlabel("$r$ [kpc]")
axes[1].set_ylabel(r"$v_c$ [${\rm km}\,{\rm s}^{-1}$]")

fig.tight_layout()

### Compare surface density to measurements

In [ ]:
from scipy.integrate import quad

In [ ]:
def func(z, R, pot, axis=0):
    q = np.array([0, 0, z])
    q[axis] = R
    return pot.density(q).value[0]

Total surface mass density in stars from [Bovy 2012](http://iopscience.iop.org/article/10.1088/0004-637X/751/2/131/pdf)

In [ ]:
Rs = np.linspace(4, 10, 16)
surf_dens = np.zeros_like(Rs)

for i,R in enumerate(Rs):
    _surf_dens,_ = quad(func, -1.1, 1.1, args=(R, pot_no_halo))
    surf_dens[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value

In [ ]:
plt.figure(figsize=(5,5))
plt.semilogy(Rs, surf_dens, marker='', linestyle='-')
plt.semilogy(Rs, 1.2E3*np.exp(-Rs/2.5), marker='', linestyle='-')
plt.scatter(8., 38) # stars
plt.scatter(8., 51) # stars+gas, according to Bovy and Rix
plt.xlim(4, 10)
plt.ylim(10, 1000)
plt.xlabel(r"$R$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

Surface mass densities read off of Fig. 3 in [Portail 2016](https://arxiv.org/pdf/1608.07954v2.pdf)

In [ ]:
xy_Rs = np.logspace(-2, 1.4, 32)
surf_dens_x = np.zeros_like(xy_Rs)
surf_dens_y = np.zeros_like(xy_Rs)

for i,R in enumerate(xy_Rs):
    _surf_dens,_ = quad(func, -1.1, 1.1, args=(R, pot_no_halo, 0))
    surf_dens_x[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value
    
    _surf_dens,_ = quad(func, -1.1, 1.1, args=(R, pot_no_halo, 1))
    surf_dens_y[i] = (_surf_dens*u.Msun/u.kpc**2).to(u.Msun/u.pc**2).value

These points were read off the plot for just the total baryonic

In [ ]:
x = [0, 2, 4, 6, 8] * u.kpc
x_sdens = ([4E9, 1E9, 5.5E8, 1E8, 4E7] * u.Msun/u.kpc**2).to(u.Msun/u.pc**2)

y = [0, 1, 2, 4, 6, 8] * u.kpc
y_sdens = ([4E9, 8E8, 2.2E8, 2E8, 1E8, 4E7] * u.Msun/u.kpc**2).to(u.Msun/u.pc**2)

plt.figure(figsize=(7,5))

plt.semilogy(xy_Rs, surf_dens_x, marker='', ls='-', color='r')
plt.semilogy(-xy_Rs, surf_dens_x, marker='', ls='-', color='r')
plt.semilogy(xy_Rs, surf_dens_y, marker='', ls='-', color='b')
plt.semilogy(-xy_Rs, surf_dens_y, marker='', ls='-', color='b')

plt.semilogy(x, x_sdens, marker='o', color='r', ls='none', mec='k', mew=1.)
plt.semilogy(y, y_sdens, marker='o', color='b', ls='none', mec='k', mew=1.)
plt.semilogy(-x, x_sdens, marker='o', color='r', ls='none', mec='k', mew=1.)
plt.semilogy(-y, y_sdens, marker='o', color='b', ls='none', mec='k', mew=1.)

plt.xlim(-8.1, 8.1)
plt.ylim(1E1, 1E4)
plt.xlabel(r"$x,y$ [kpc]")
plt.ylabel(r"$\Sigma(R)$ [${\rm M}_\odot \, {\rm pc}^{-2}$]")

In [ ]:
def density_on_grid(density, grid_lim=(-10,10), ngrid=128):
    grid = np.linspace(grid_lim[0], grid_lim[1], ngrid)
    xyz = np.vstack(map(np.ravel, np.meshgrid(grid,grid,grid)))

    val = np.zeros((ngrid*ngrid*ngrid,))
    val = density(xyz).value
    val[np.isnan(val)] = val[np.isfinite(val)].max()
    
    gridx = xyz[0].reshape(ngrid,ngrid,ngrid)[:,:,0]
    gridy = xyz[1].reshape(ngrid,ngrid,ngrid)[:,:,0]
    
    return gridx, gridy, val

In [ ]:
ngrid = 128
# xx,yy,dens3d = density_on_grid(pot_no_halo.density, ngrid=ngrid, grid_lim=(-8,8))
xx,yy,dens3d = density_on_grid(pot.density, ngrid=ngrid, grid_lim=(-8,8))
dens3d = dens3d.reshape(ngrid,ngrid,ngrid)

In [ ]:
levels = np.logspace(7., 10, 16)
norm = mpl.colors.LogNorm(vmin=levels.min(), vmax=levels.max())
# norm = mpl.colors.Normalize(vmin=levels.min(), vmax=levels.max())

# Used for integration:
_x = np.linspace(xx.min(), xx.max(), xx.shape[0])

fig, axes = plt.subplots(1, 2, figsize=(10,4.8))

dens2d = simps(dens3d, x=_x, axis=2)
axes[0].pcolormesh(xx, yy, dens2d, cmap='magma', norm=norm)
axes[0].contour(xx, yy, dens2d, 
                levels=levels,
                colors='k', alpha=0.75, linewidths=1)

dens2d = simps(dens3d, x=_x, axis=0)
axes[1].pcolormesh(xx, yy, dens2d.T, cmap='magma', norm=norm)
axes[1].contour(xx, yy, dens2d.T,
                levels=levels,
                colors='k', alpha=0.75, linewidths=1)